<a href="https://colab.research.google.com/github/DSlaughter01/election-stability/blob/main/electoral_systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A cleaner attempt at creating an election class

In [5]:
# Import and install necessary modules
%pip install igraph
%pip install cairocffi

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import igraph as ig
# import cairocffi
from statistics import mode
from collections import Counter
from math import floor, ceil
from heapq import nlargest
from random import choices, choice, shuffle
import seaborn as sns
import time
import warnings


Note: you may need to restart the kernel to use updated packages.


In [6]:
class Election():

  def __init__(self, n = 500, part = 3, elecs = 100, iters = 5000, dist = 10,
               pref = 1, zeal = 0.05, alpha = 0.05, t = 0, track = False):

    self.n = n          # The number of nodes in the network
    self.part = part    # The number of parties available to vote for
    self.elecs = elecs  # The number of elections to perform
    self.iters = iters  # Number of interactions between voters
    self.dist = dist    # Number of electoral districts (= seats available for now)
    self.pref = pref    # Number of voter party preferences
    self.zeal = zeal    # Probability that a node in the zealot party is a zealot
    self.alpha = alpha  # Noise rate
    self.t = t          # Whether it is the first election (t = 0) or not
    self.track = track  # Whether tracking of every iteration is desired

    # The final number of votes and seats in each election - to be
    self.votes = np.empty((self.part, self.elecs))
    self.seats = np.empty((self.part, self.elecs))

    # Tracking votes over iterations of the voter model if desired
    self.vm_tracker = []
    if self.track:
      self.vm_tracker = np.empty((self.elecs, self.iters, self.part))

    # Check parameter values
    if self.zeal > 1 or self.zeal < 0: 
      raise ValueError("Please enter a zeal value between 0 and 1")
    if self.alpha > 1 or self.alpha < 0: 
      raise ValueError("Please enter an alpha value between 0 and 1")



  def init_graph(self):
    """
    Function to initialise a graph, and allocate zealots and opinions
    """

    a = self.dist

    # First generate an SBM
    # The preference matrix - symmetric, stronger on the diagonal, random
    lt = np.tril(np.random.rand(10, 10), k = -1)
    ut = np.transpose(lt)
    diags = np.diag(np.random.rand(10))
    pref_mat = 0.25*(lt + ut + diags) + 0.75*(np.identity(10))

    # Generate the SBM from this preference matrix
    split = np.array_split(range(self.n), a)
    block_sizes = [len(i) for i in split]
    start_graph = ig.Graph.SBM(n = self.n,
                               pref_matrix = list(pref_mat),
                               block_sizes = block_sizes)
    
    # Assign a State to each node from self.part states
    # Take into account the number of preferences

    # Add interaction nodes
    if self.zeal == 0:
      start_graph.vs()["Interaction"] = "Normal"

    else:
      for i in start_graph.vs():
        if i["State"] == 0:
          i["Interaction"] = choices(["Normal", "Zealot"], 
                                     weights = [round(100*(1 - self.zeal)), round(100*self.zeal)],
                                     k = 1)[0]
        else:
          i["Interaction"] = "Normal"
    
    for i in start_graph.vs():
      shuffled_pref = shuffle(self.pref)

      for j in range(self.pref):
        state = f"State_{j + 1}"
        i[state] = shuffled_pref[i]

    # TODO
    # Consider how is best to assign zealots



  def voter_model(self, track = False):
    """
    Function to perform iterations of the voter model
    """
    # TODO
    # First code voter model when not tracking
    # Then extend it to tracking

    # Generate an initial graph
    self.init_graph()

    # Dot notation removed for future convenience
    current_graph, p, n = self.curr, self.part, self.n
    randint, rand, vert = np.random.randint, np.random.random, current_graph.vs

    # Run the voter model for iters iterations
    for iter in range(self.iters):

      # Select a random node
      i = randint(0, n)
      active = vert[i]

      # Get the number of neighbours
      neigh_list = active.neighbors()

      # Select a neighbour at random
      j = randint(0, len(neigh_list))

      # Match the active State to the neighbour State if not zealot
      if active["Interaction"] == "Normal": 
        active["State"] = neigh_list[j]["State"]

      # Implement random change for node i if Interaction is normal
      k = rand()
      if k <= self.alpha and active["Interaction"] == "Normal":
        active["State"] = choice([i for i in range(0,self.no_parties) if i != active["State"]])

      # Ensure every party is listed even if they have 0 votes
      zerokeys, zerovalues = range(self.no_parties), [0]*self.no_parties
      zerodict = dict(zip(zerokeys, zerovalues))

      # Data for plotting
      vote_counts = {}
      for k in current_graph.vs()["State"]:
        vote_counts[k] = vote_counts.get(k, 0) + 1
      vote_counts = {**zerodict, **vote_counts}

      # The final stage is the final number of votes
      if iter == self.iters - 1:
        self.votes.append(list(vote_counts.values()))

      # TODO
      # Check this works, given the new way of appending

  def vm_plot(self):
    """
    Function to plot trajectories of the voter model
    """
    pass

  def fptp(self):
    """
    Function to call a runthrough of the voter model, and
    perform a First Past the Post (FPTP) election.
    """
    pass


  def pr(self):
    """
    Function to call a runthrough of the voter model, and
    perform a Proportional Representation (PR) election.
    """
    pass

  # Two-round system - may be difficult allocating political opinions
  def two_round(self):
    pass

  # STV - need to allocate multiple opinions
  def stv(self):

    # The following is not necessarily for any type of election,
      # but more how the voter model could handle multiple opinions
    # Opinion allocation:
      # Take user input/a parameter for how many preferences
      # Make a list of the preferences in a range()
      # Allocate the first, remove it from the list, repeat until empty list
        # Assign an interaction (normal or zealot) to each node

    pass

  # One to plot how the underlying network looks
  def net_plot(self):
    """
    Function to plot the network, including "Interaction"
    and "State" modules
    """
    pass

  # One to plot histograms
  def plot_hist(self):
    pass



  # How can we efficiently allocate space, and reduce time?

In [ ]:
g = Election(pref = 3)
g.init_graph()
# Print 

range(0, 3)
